In [ ]:
import requests
import requests.cookies
import json
import os
import ssl
import certifi
import websockets
import re


class TradeRepublicApi:
    HOST = "https://api.traderepublic.com"
    WSS_HOST = "wss://api.traderepublic.com"
    HEADERS = {
        "User-Agent": "Mozilla/5.0 (Linux; Android 10; SM-G960F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.181 Mobile Safari/537.36",
    }

    def __init__(self, phone: str, pin: str):
        self.phone = phone
        self.pin = pin
        self.session = requests.Session()
        self.session.headers.update(self.HEADERS)

    def login(self):
        response = self.session.post(
            url=f"{self.HOST}/api/v1/auth/web/login",
            json={"phoneNumber": self.phone, "pin": self.pin},
        )

        assert response.status_code == 200, response.text

        r = response.json()

        assert "processId" in r, response.text

        self.process_id = r["processId"]
        print(f"Login process ID: {self.process_id}")
        

    def enter_2fa_code(self, code: str):
        url = f"{self.HOST}/api/v1/auth/web/login/{self.process_id}/{code}"
        response = self.session.post(
            url=url,
        )

        assert response.status_code == 200, response.text

        print(response)
        print(response.text)
    
    def refresh_session(self):
        response = self.session.get(
            url=f"{self.HOST}/api/v1/auth/session",
        )

        assert response.status_code == 200, response.text

        print(response)
        print(response.text)

        self._save_session()

    def is_logged_in(self):
        response = self.session.get(url=f"{self.HOST}/api/v2/auth/account")

        return response.status_code == 200
    

    def account(self):
        response = self.session.get(url=f"{self.HOST}/api/v2/auth/account")

        assert response.status_code == 200, response.text

        print(response.text)
        
    async def connect_websocket(self):
        ssl_context = ssl.create_default_context(cafile=certifi.where())
        additional_headers = {
            "Cookie": "; ".join(
                [
                    f"{cookie.name}={cookie.value}"
                    for cookie in self.session.cookies
                    if cookie.domain.endswith("traderepublic.com")
                ]
            ),
        }

        self.ws = await websockets.connect(
            uri=f"{self.WSS_HOST}",
            additional_headers=additional_headers,
            ssl=ssl_context,
        )
        await self.ws.send(
            f"connect 31 {json.dumps({
                "locale": "en",
                "platformId": "webtrading",
                "platformVersion": "chrome - 131.0.0",
                "clientId": "app.traderepublic.com",
                "clientVersion": "3.130.0",
            })}"
        )
        response = await self.ws.recv()        
        assert "connected" == response, response     
        
    async def timeline_transactions(self):
        await self.ws.send(
            f"sub 165 {json.dumps({'type': 'timelineTransactions', "token": self.process_id})}"
        )
        response = await self.ws.recv()
        
        json_string = re.search(r"\d+ \w+ (.*)", response).group(1)
        transactions = json.loads(json_string)
        
        assert "errors" not in transactions, transactions
        assert "items" in transactions, transactions
        
        response = await self.ws.recv() # close message
        
        for transaction in transactions["items"]:
            assert "action" in transaction, transaction
            transaction["actionJson"] = transaction["action"]
            action = transaction["actionJson"]
            assert "payload" in action, action
            payload = action["payload"]
            
            print(payload)
            await self.ws.send(
                f"sub 125 {json.dumps({'type': 'timelineDetailV2', 'id': payload})}"
            )
            response = await self.ws.recv()
            
            print(response)
            json_string = re.search(r"\d+ \w+ (.*)", response).group(1)
            
            action = json.loads(json_string)
            
            transaction["action"] = action
            
            response = await self.ws.recv() # close message
            
                    
        
# sub 125 {type: "timelineDetailV2", id: "4c3e2853-d010-44b8-9240-b2e3bcd2bb7b"}
# id
# : 
# "4c3e2853-d010-44b8-9240-b2e3bcd2bb7b"
# type
# : 
# "timelineDetailV2"
        
        
        return transactions

secrets = json.loads(open("secrets.json", "r", encoding="utf-8").read())
phone = secrets["phone"]
pin = secrets["pin"]

api = TradeRepublicApi(phone, pin)

api.login()
code = input("Enter 2FA code: ")
api.enter_2fa_code(code)

await api.connect_websocket()
transactions = await api.timeline_transactions()

# write to file
with open("transactions.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(transactions, indent=4))

In [ ]:
import json
import pandas as pd

# load from file
with open("transactions.json", "r", encoding="utf-8") as f:
    transactions = json.load(f)
    
print(transactions["cursors"])
print(transactions["startingTransactionId"])

df = pd.DataFrame(transactions["items"])

# change amount column to amountJson
df["amountJson"] = df["amount"]

# insert after amount column a new column called currency
df.insert(df.columns.get_loc("amount") + 1, "currency", None)
df["currency"] = df["amountJson"].apply(lambda x: x["currency"])
df["amount"] = df["amountJson"].apply(lambda x: str(x["value"]).replace(".", ","))


currencies = set(df["currency"])
assert len(currencies) == 1, f"Expected one currency, got {currencies}"

# write to file
df.to_csv("transactions.csv", index=False)

df

In [ ]:
# TRADE_INVOICE, GIFTER_TRANSACTION, SAVINGS_PLAN_INVOICE_CREATED
trades = df[df["eventType"].isin(["TRADE_INVOICE", "SAVINGS_PLAN_INVOICE_CREATED"])]
print(trades.iloc[0]["action"])

info = pd.DataFrame()
for action in trades['action'].values:
    info_section = action["sections"][0]
    overview_section = action["sections"][1]
    document_section = None
    for section in action["sections"]:
        if section["title"] == "Documents":
            document_section = section
    
    timestamp = info_section["data"]["timestamp"]
    asset = overview_section["data"][2]["detail"]["text"]
    
    for document in document_section["data"]:
        title = document["title"]
        detail = document["detail"]
        
        if "action" in document:
            url = document["action"]["payload"]
            id_ = document["id"]
        
            # download the url
            response = requests.get(url)
            assert response.status_code == 200
            
            os.makedirs(f"pdfs/{asset}", exist_ok=True)            
            filename = f"{detail} - {title} - {id_}.pdf"
            with open(f"pdfs/{asset}/{filename}", "wb") as f:
                f.write(response.content)
info

In [ ]:
import gspread
gc = gspread.service_account()

sh = gc.open("Finanzen")

ws = sh.worksheet("Trade Republic")

# write the columns to the worksheet

In [ ]:
ws.clear()

cells = [df.columns.values.tolist()] + df.values.tolist()
cells = df.values.tolist()
# convert all values to string if they are not null
for i, row in enumerate(cells):
    for j, cell in enumerate(row):
        if cell:
            cells[i][j] = str(cell)
        else:
            cells[i][j] = ""
ws.update(cells, "A1", value_input_option="USER_ENTERED")

# set the G column to be currency euro
ws.format("G1:G15", {"numberFormat": {"type": "CURRENCY", "pattern": "#,##0.00 €"}})